In [2]:
! pip install syft 

In [0]:
! pip install tf_encrypted

###  Federated Learning CPU

In [0]:
import torch
#torch.set_default_tensor_type(torch.cuda.FloatTensor)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
#torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [4]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

W0903 09:28:43.631935 140105902241664 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0903 09:28:43.650582 140105902241664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
NUM_CLIENTS = 3383
SAMPLES = 10

In [0]:
clients=[]
for i in range(NUM_CLIENTS):
        clients.append(sy.VirtualWorker(hook, id="c "+str(i)))

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 20
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [13]:
train_dataset = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))]))

0it [00:00, ?it/s]

9920512it [00:02, 4292693.34it/s]                             


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 68774.94it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1129398.23it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 25752.00it/s]            

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [29]:
A = range(11)
A[2:8]

range(2, 8)

In [38]:
train_dataset[0].targets

AttributeError: ignored

In [0]:
train_idx = int(len(train_dataset)/3383)
print(train_idx)

In [0]:
A = list(train_dataset)[:train_idx*SAMPLES]
train,labels=[],[]
for i in range(SAMPLES):
  L,K = [],[]
  for j in range(train_idx):
    L.append(A[SAMPLES*i+j][0])
    K.append(A[SAMPLES*i+j][1])
  train.append(L)
  labels.append(K)
  

In [55]:
train = torch.tensor(train)

TypeError: ignored

In [52]:
# Sending datasets to virtual workers
fed_train_dataset = []
for i in range(SAMPLES):
  print(i)
  fed_train_dataset.append(sy.BaseDataset(train[i],labels[i]).send("c "+str(i)))
  


0


AttributeError: ignored

In [0]:
federated_train_loader = sy.FederatedDataLoader(train)

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [8]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate(clients), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


0it [00:00, ?it/s]

9920512it [00:01, 8802195.80it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 134108.24it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2203807.57it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 50919.69it/s]            


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [7]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data',split='digits', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate(clients), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)




TypeError: ignored

In [9]:
test_loader = torch.utils.data.DataLoader(
    datasets.EMNIST('../data',split='digits' ,train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

RuntimeError: ignored

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc = nn.Linear(784,10)
    
  def forward(self,x):
    x = x.view(-1,784)
    x = self.fc(x)
    return F.log_softmax(x,dim=1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    i = 0
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        if i < 11:
          i+=1
          model.send(data.location) # <-- NEW: send the model to the right location
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = F.nll_loss(output, target)
          loss.backward()
          optimizer.step()
          model.get() # <-- NEW: get the model back
          #if batch_idx % args.log_interval == 0:
          loss = loss.get() # <-- NEW: get the loss back
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                  100. * batch_idx / len(federated_train_loader), loss.item()))
        else:
          break
        
      

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def train_on_batches(worker, batches, model_in, device, lr,epoch,clients_mem):
    """Train the model on the worker on the provided batches
    Args:
        worker(syft.workers.BaseWorker): worker on which the
        training will be executed
        batches: batches of data of this worker
        model_in: machine learning model, training will be done on a copy
        device (torch.device): where to run the training
        lr: learning rate of the training steps
    Returns:
        model, loss: obtained model and loss after training
    """
    model = model_in.copy()
    optimizer = optim.SGD(model.parameters(), lr=lr)  # TODO momentum is not supported at the moment

    model.train()
    model.send(worker)
    loss_local = False
    D = {"alice":0,"bob":1,"charlie":2}
    i = D[worker.id]
    for batch_idx, (data, target) in enumerate(batches):
        loss_local = False
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = f.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            loss = loss.get()  # <-- NEW: get the loss back
            loss_local = True
            print(
                "Epoch {} Train Worker {}: [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    worker.id,
                    batch_idx,
                    len(batches),
                    100.0 * batch_idx / len(batches),
                    loss.item(),
                )
            )

    if not loss_local:
        loss = loss.get()  # <-- NEW: get the loss back
    model.get()  # <-- NEW: get the model back
    clients_mem[i] += 2*model_size(model)
    return model, loss


def get_next_batches(fdataloader: sy.FederatedDataLoader, nr_batches: int):
    """retrieve next nr_batches of the federated data loader and group
    the batches by worker
    Args:
        fdataloader (sy.FederatedDataLoader): federated data loader
        over which the function will iterate
        nr_batches (int): number of batches (per worker) to retrieve
    Returns:
        Dict[syft.workers.BaseWorker, List[batches]]
    """
    batches = {}
    for worker_id in fdataloader.workers:
        worker = fdataloader.federated_dataset.datasets[worker_id].location
        batches[worker] = []
    try:
        for i in range(nr_batches):
            next_batches = next(fdataloader)
            for worker in next_batches:
                batches[worker].append(next_batches[worker])
    except StopIteration:
        pass
    return batches


def train(model, device, federated_train_loader, lr, federate_after_n_batches,epoch,clients_mem):
    model.train()

    nr_batches = federate_after_n_batches

    models = {}
    loss_values = {}

    iter(federated_train_loader)  # initialize iterators
    batches = get_next_batches(federated_train_loader, nr_batches)
    counter = 0

    while True:
        print(
            "Starting training round, batches [{}, {}]".format(counter, counter + nr_batches)
        )
        data_for_all_workers = True
        for worker in batches:
            curr_batches = batches[worker]
            if curr_batches:
                models[worker], loss_values[worker] = train_on_batches(
                    worker, curr_batches, model, device, lr,epoch,clients_mem
                )
            else:
                data_for_all_workers = False
        counter += nr_batches
        if not data_for_all_workers:
            print("At least one worker ran out of data, stopping.")
            break

        model = utils.federated_avg(models)
        batches = get_next_batches(federated_train_loader, nr_batches)
    return model


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += f.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print("\n")
    accuracy = 100.0 * correct / len(test_loader.dataset)
    print(
        "Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), accuracy
        )
    )

In [12]:
import time
#%%time
start = time.time()
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")
    
end = time.time()
print(end-start)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.533023
Train Epoch: 1 [20/60000 (0%)]	Loss: 2.231701
Train Epoch: 1 [40/60000 (0%)]	Loss: 2.097592
Train Epoch: 1 [60/60000 (0%)]	Loss: 1.941655
Train Epoch: 1 [80/60000 (0%)]	Loss: 1.561559
Train Epoch: 1 [100/60000 (0%)]	Loss: 1.522289
Train Epoch: 1 [120/60000 (0%)]	Loss: 1.483977
Train Epoch: 1 [140/60000 (0%)]	Loss: 1.816720
Train Epoch: 1 [160/60000 (0%)]	Loss: 1.581472
Train Epoch: 1 [180/60000 (0%)]	Loss: 1.513144
Train Epoch: 1 [200/60000 (0%)]	Loss: 1.491247

Test set: Average loss: 1.4848, Accuracy: 5787/10000 (58%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.291868
Train Epoch: 2 [20/60000 (0%)]	Loss: 1.304456
Train Epoch: 2 [40/60000 (0%)]	Loss: 1.289812
Train Epoch: 2 [60/60000 (0%)]	Loss: 1.031413
Train Epoch: 2 [80/60000 (0%)]	Loss: 0.841145
Train Epoch: 2 [100/60000 (0%)]	Loss: 0.802011
Train Epoch: 2 [120/60000 (0%)]	Loss: 0.782497
Train Epoch: 2 [140/60000 (0%)]	Loss: 1.243627
Train Epoch: 2 [160/60000 (0%)]	Loss: 1.060057
Train Epoch

### SPLIT LEARNING CPU

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 27476627.61it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 445880.03it/s]
  1%|          | 16384/1648877 [00:00<00:11, 142108.94it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7442404.28it/s]                            
8192it [00:00, 182759.71it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.max_pool2d(x, 2, 2)

In [0]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
models = [Net1(),Net1()]
models[0] = models[0].send(bob)
models[1] = models[1].send(alice)


opt1 = optim.SGD(params=models[0].parameters(),lr=0.1)
opt2 = optim.SGD(params=models[1].parameters(),lr=0.1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, targs) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        #IF ON DATA LOCATION TO GET THE RIGHT MODEL
        if data.location.id == 'bob':
          mod_c,opt_c = models[0], opt1
        else : 
          mod_c,opt_c = models[1], opt2
          
       # 1) erase previous gradients (if they exist)
        optimizer.step()
        opt_c.step()
        opt_c.zero_grad()
        optimizer.zero_grad()
        
        
        #model.send(data.location) # <-- NEW: send the model to the right location
        #target= tar.get()
        tg_copy = targs.copy()
        target = tg_copy.get()
        data, target = data.to(device), target.to(device)
        #optimizer.zero_grad()
        # 2) make a prediction until cut layer (client location)
        pred_c = mod_c(data)
        copy = pred_c.copy()
        # 3) get this to the server 
        inp = copy.get()

        # 4) make prediction with second part of the model (server location)
        pred = model(inp)

        # 5) calculate how much we missed 
        #print(pred.size(),target.size())
        loss = F.nll_loss(pred, target)
        loss.backward()
        
        gradient = inp.grad
        gradient = gradient.send(data.location)
        pred_c.backward(gradient)
        
       #target = target.send(data.location)
        #optimizer.step()
        #model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            #loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    M1 = models[0].copy()
    M2 = models[1].copy()
    M1 = M1.get()
    M2 = M2.get()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(M1(data))
            #output2 = model(M2(data))
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
%%time
model = Net2().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.302813
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.123377
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.199097
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.594879
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.559763
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.425805
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.171876
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.300623
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.436527
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.656055
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.144281
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.374599
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.114359
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.329657
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.041824
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.124439
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.506185
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.249304
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.408869
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.161326
Train Epoch:

###  Federated Learning GPU

In [0]:
! git clone https://github.com/OpenMined/PySyft.git

Cloning into 'PySyft'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 27077 (delta 9), reused 16 (delta 6), pack-reused 27052
Receiving objects: 100% (27077/27077), 31.68 MiB | 19.72 MiB/s, done.
Resolving deltas: 100% (17646/17646), done.


In [0]:
ls

PySyft/  sample_data/


In [0]:
! pip insg

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

ModuleNotFoundError: ignored

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

if use_cuda:
        # TODO Quickhack. Actually need to fix the problem moving the model to CUDA\n",
        torch.set_default_tensor_type(torch.cuda.FloatTensor)
torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")
#device = torch.device("cpu")

#kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs = {'num_workers': 0, 'pin_memory': False} if use_cuda else {}

IndexError: ignored

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


0it [00:00, ?it/s]

9920512it [00:01, 8422041.44it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 137291.50it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2231027.05it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 52421.36it/s]            


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.305134
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.156802
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.896600
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.440342
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.867032
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.654484
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.593135
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.455443
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.370512
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.304318
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.313664
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.369745
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.238009
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.187349
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.524126
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.224937
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.174450
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.201018
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.207578
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.311946
Train Epoch:

### Split NN config 2

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 18943904.97it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 318986.76it/s]                           
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5231109.26it/s]                           
8192it [00:00, 132621.61it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.max_pool2d(x, 2, 2)

In [0]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
models = [Net1(),Net1()]
models[0] = models[0].send(bob)
models[1] = models[1].send(alice)


opt1 = optim.SGD(params=models[0].parameters(),lr=0.1)
opt2 = optim.SGD(params=models[1].parameters(),lr=0.1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, targs) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        #IF ON DATA LOCATION TO GET THE RIGHT MODEL
        if data.location.id == 'bob':
          mod_c,opt_c = models[0], opt1
        else : 
          mod_c,opt_c = models[1], opt2
          
       # 1) erase previous gradients (if they exist)
        optimizer.step()
        opt_c.step()
        opt_c.zero_grad()
        optimizer.zero_grad()
        
        
        #model.send(data.location) # <-- NEW: send the model to the right location
        #target= tar.get()
        #tg_copy = targs.copy()
        #target = tg_copy.get()
        
        data, target = data.to(device), targs.to(device)
        #optimizer.zero_grad()
        # 2) make a prediction until cut layer (client location)
        pred_c = mod_c(data)
        copy = pred_c.copy()
        # 3) get this to the server 
        inp = copy.get()

        # 4) make prediction with second part of the model (server location)
        pred = model(inp)
        cop = pred.copy()
        out = cop.send(data.location)
        # 5) calculate how much we missed 
        #print(pred.size(),target.size())
        loss = F.nll_loss(out, target)
        loss.backward()
        grad = out.grad
        #grad = grad.clone().get()
        grad1 = grad.copy()
        grad2 = grad1.get()
        print("grad:",grad2.shape)
        print("pred",pred.shape)
        pred.backward(grad2)
        
        
        gradient = inp.grad
        print('inp',inp.shape)
        print('gradient',gradient.shape)
        gradient = gradient.send(data.location)
        print("pred_c",pred_c.shape)
        print('gradient',gradient.shape)
        pred_c.backward(gradient)
        
        
       #target = target.send(data.location)
        #optimizer.step()
        #model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            #loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    M1 = models[0].copy()
    M2 = models[1].copy()
    M1 = M1.get()
    M2 = M2.get()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(M1(data))
            #output2 = model(M2(data))
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
%%time
model = Net2().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

grad: torch.Size([64, 10])
pred torch.Size([64, 10])
inp torch.Size([64, 20, 12, 12])
gradient torch.Size([64, 20, 12, 12])
pred_c torch.Size([64, 20, 12, 12])
gradient torch.Size([64, 20, 12, 12])


RuntimeError: ignored

## TESTS


In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [0]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.max_pool2d(x, 2, 2)

In [0]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
models = [Net1(),Net1()]
models[0] = models[0].send(bob)
models[1] = models[1].send(alice)


opt1 = optim.SGD(params=models[0].parameters(),lr=0.1)
opt2 = optim.SGD(params=models[1].parameters(),lr=0.1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, targs) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        #IF ON DATA LOCATION TO GET THE RIGHT MODEL
        if data.location.id == 'bob':
          mod_c,opt_c = models[0], opt1
        else : 
          mod_c,opt_c = models[1], opt2
          
       # 1) erase previous gradients (if they exist)
        optimizer.step()
        opt_c.step()
        opt_c.zero_grad()
        optimizer.zero_grad()
        
        
        #model.send(data.location) # <-- NEW: send the model to the right location
        #target= tar.get()
        tg_copy = targs.copy()
        target = tg_copy.get()
        data, target = data.to(device), target.to(device)
        #optimizer.zero_grad()
        # 2) make a prediction until cut layer (client location)
        pred_c = mod_c(data)
        copy = pred_c.copy()
        # 3) get this to the server 
        inp = copy.get()

        # 4) make prediction with second part of the model (server location)
        pred = model(inp)

        # 5) calculate how much we missed 
        #print(pred.size(),target.size())
        loss = F.nll_loss(pred, target)
        loss.backward()
        
        gradient = inp.grad
        gradient = gradient.send(data.location)
        pred_c.backward(gradient)
        
       #target = target.send(data.location)
        #optimizer.step()
        #model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            #loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    M1 = models[0].copy()
    M2 = models[1].copy()
    M1 = M1.get()
    M2 = M2.get()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(M1(data))
            #output2 = model(M2(data))
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
%%time
model = Net2().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.302813
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.123377
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.199097
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.594879
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.559763
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.425805
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.171876
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.300623
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.436527
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.656055
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.144281
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.374599
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.114359
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.329657
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.041824
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.124439
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.506185
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.249304
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.408869
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.161326
Train Epoch:

###  Federated Learning CPU

In [0]:
import torch
#torch.set_default_tensor_type(torch.cuda.FloatTensor)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
#torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [0]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

W0619 15:48:01.123609 140702398183296 hook.py:97] Torch was already hooked... skipping hooking process


In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.305134
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.156802
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.896587


KeyboardInterrupt: ignored

### SPLIT LEARNING CPU

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 28501590.63it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 450556.00it/s]
  1%|          | 16384/1648877 [00:00<00:11, 142488.75it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7114838.39it/s]                            
8192it [00:00, 182204.38it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.max_pool2d(x, 2, 2)

In [0]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
models = [Net1(),Net1()]
models[0] = models[0].send(bob)
models[1] = models[1].send(alice)


opt1 = optim.SGD(params=models[0].parameters(),lr=0.1)
opt2 = optim.SGD(params=models[1].parameters(),lr=0.1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, targs) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        #IF ON DATA LOCATION TO GET THE RIGHT MODEL
        if data.location.id == 'bob':
          mod_c,opt_c = models[0], opt1
        else : 
          mod_c,opt_c = models[1], opt2
          
       # 1) erase previous gradients (if they exist)
        optimizer.step()
        opt_c.step()
        opt_c.zero_grad()
        optimizer.zero_grad()
        
        
        #model.send(data.location) # <-- NEW: send the model to the right location
        #target= tar.get()
        tg_copy = targs.copy()
        target = tg_copy.get()
        data, target = data.to(device), target.to(device)
        #optimizer.zero_grad()
        # 2) make a prediction until cut layer (client location)
        pred_c = mod_c(data)
        copy = pred_c.copy()
        # 3) get this to the server 
        inp = copy.get()

        # 4) make prediction with second part of the model (server location)
        pred = model(inp)

        # 5) calculate how much we missed 
        #print(pred.size(),target.size())
        loss = F.nll_loss(pred, target)
        loss.backward()
        
        gradient = inp.grad
        print()
        gradient = gradient.view(inp.shape)
        G = gradient.send(data.location)
        print(G.shape)
        print(pred_c.shape)
        pred_c.backward(G)
        print(data.grad)
        #target = target.send(data.location)
        #optimizer.step()
        #model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            #loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    M1 = models[0].copy()
    M2 = models[1].copy()
    M1 = M1.get()
    M2 = M2.get()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(M1(data))
            #output2 = model(M2(data))
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
%%time
model = Net2().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")


torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None
Train Epoch: 1 [0/60032 (0%)]	Loss: 2.312780

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])
None

torch.Size([64, 20, 12, 12])
torch.Size([64, 20, 12, 12])


KeyboardInterrupt: ignored

In [0]:
a = torch.tensor([[1.,1.],[1.,1.]])

In [0]:
b = torch.tensor([2.,2.,2.,2.])

In [0]:
b.shape

torch.Size([4])

In [0]:
a.shape

torch.Size([2, 2])

In [0]:
a.view(b.shape)

tensor([1., 1., 1., 1.])

In [0]:
a

tensor([[1., 1.],
        [1., 1.]])